In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-11-05"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
16258,2024-11-05,França Pro B,16:00,Fos-sur-Mer,Denain-Voltaire,1.42,2.57,154.5,1.86,1.80,-5.5,1.92,1.75,rN2kS5Hl,0.704225,0.389105,0.537634,0.555556,0.093330,192.556,37.722710,0.195905,0.6,1.341641,2.236068,252.72,2.654,0.600566,0.226287,-18.0,125.028,20.018100,0.160109,1.2,1.643168,1.369306,133.60,1.518,0.264235,0.174068,12.0,78,80,3.24,1.67,125.556,211.256,0.407605,0.023184,0.065509,-0.44,-0.088,-4.772727,0.658896,0.6,-0.058896,2.92,0.584,2.688356,0.397615,0.4,0.002385
16259,2024-11-05,França Pro B,16:00,Chartres,Evreux,1.45,2.48,167.5,1.83,1.83,-5.5,1.98,1.70,C2f6ONGD,0.689655,0.403226,0.546448,0.546448,0.092881,488.666,232.020470,0.474804,0.6,1.341641,2.236068,392.63,6.010,2.664010,0.443263,-56.0,266.108,130.725736,0.491251,0.6,1.341641,2.236068,136.80,3.800,2.073861,0.545753,-45.0,79,76,4.97,1.80,0.000,262.810,0.370646,0.000000,0.107603,0.00,0.000,inf,0.000000,0.0,0.000000,-1.63,-0.326,-4.539877,0.315479,0.2,-0.115479
16260,2024-11-05,França Pro B,16:00,Chalons-Reims,Ada Blois,3.52,1.24,159.5,1.80,1.86,6.5,1.95,1.72,jm6cQqn1,0.284091,0.806452,0.555556,0.537634,0.090543,192.648,79.785294,0.414151,1.2,1.643168,1.369306,156.24,2.444,0.854125,0.349478,-33.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,84,75,1.86,1.16,129.596,0.000,0.677396,0.023184,0.088629,2.17,0.434,5.806452,0.725017,0.9,0.174983,0.00,0.000,inf,0.000000,0.0,0.000000
16261,2024-11-05,Finlândia Korisliiga,13:30,Helsinki Seagulls,Kobrat,1.03,8.30,167.5,1.81,1.85,-16.5,1.96,1.72,fucJfEGj,0.970874,0.120482,0.552486,0.540541,0.091356,123.444,20.812919,0.168602,2.4,1.341641,0.559017,131.58,1.306,0.175727,0.134554,70.0,226.634,131.308751,0.579387,1.2,1.643168,1.369306,110.08,2.774,1.451665,0.523311,-12.0,102,64,1.29,1.72,118.836,444.270,1.101965,0.015456,0.092231,0.48,0.096,0.312500,0.769113,0.7,-0.069113,-5.00,-1.000,-7.300000,0.336133,0.4,0.063867
16262,2024-11-05,Finlândia Korisliiga,13:30,BC Nokia,Honka,1.22,3.64,164.5,1.83,1.83,-9.5,1.91,1.76,OhznDMWH,0.819672,0.274725,0.546448,0.546448,0.094397,148.872,26.768191,0.179807,1.2,1.643168,1.369306,152.10,1.938,0.374660,0.193323,-23.0,203.448,72.298769,0.355367,0.6,1.341641,2.236068,111.15,2.650,1.059976,0.399991,-44.0,78,65,1.95,1.71,151.146,0.000,0.704197,0.000000,0.057802,1.28,0.256,0.859375,0.635138,0.7,0.064862,0.00,0.000,inf,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16314,2024-11-05,Europa Liga Dos Campeões,14:00,Galatasaray,Promitheas,1.27,3.62,163.5,1.85,1.85,-9.5,2.01,1.69,CUCigtzR,0.787402,0.276243,0.540541,0.540541,0.063645,141.876,34.922750,0.246150,1.8,1.643168,0.912871,144.57,1.696,0.529320,0.312099,-3.0,205.020,96.736541,0.471840,0.6,1.341641,2.236068,144.00,2.948,2.031470,0.689101,-30.0,79,75,1.83,1.92,132.360,222.976,0.679632,0.000000,0.122310,-0.62,-0.124,-2.177419,0.000000,0.0,0.000000,0.46,0.092,28.478261,0.000000,0.0,0.000000
16315,2024-11-05,Europa Liga Dos Campeões,16:00,Vechta,Nymburk,2.36,1.56,166.5,1.86,1.88,2.5,1.97,1.72,Gjy6gHzR,0.423729,0.641026,0.537634,0.531915,0.064754,0.000,0.000000,0.000000,0.0,0.000000,0.000000,

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

Sem jogos
